In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
from funzioni import compute_dataset_weighted
from funzioni import compute_annual_mean_weighted
from funzioni import convert_dataset_celsius
from funzioni import compute_anomaly
from funzioni import create_djf_dataset
from funzioni import compute_mean_time_season
from funzioni import compute_mean_resample_mar
from funzioni import plot_bias_atmos
from funzioni import plot_bias_tos
from funzioni import plot_mean_cluster_atmos
from funzioni import plot_bias_2_models_atmos
from funzioni import plot_zonmean
from funzioni import plot_zonmean_2_cluster
from funzioni import plot_mean_cluster_zonmean
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import plotly.express as px
import dill
import pickle
import cartopy.crs as ccrs

In [2]:
# creo un dizionario in cui andrò a mettere tutti i modelli dentro
models_orog = {}

In [3]:
#Percorso dei file
name_models_orog = '/work/users/guest/sbeltrami/prova/orog_name_ok_model.txt'
path_models_orog = '/work/users/guest/sbeltrami/prova/path_land_orog_model.txt'
path_remap_models_orog = '/work/users/guest/sbeltrami/prova/path_remap_orog_model.txt'
#path_zonal_model_models_orog = '/work/users/guest/sbeltrami/prova/path_orog_model.txt'
# Apro i 3 file in modalità di lettura
with open(name_models_orog, 'r') as file: #nome file
    # Leggo tutte le righe del file e salvo in una lista
    lines_name_models_orog = file.readlines()
with open(path_models_orog, 'r') as file: #percorso dei file .nc
    lines_path_models_orog = file.readlines()
with open(path_remap_models_orog, 'r') as file: #percorso dei file remap
    lines_path_remap_models_orog = file.readlines()
#with open(path_zonal_model_models_orog, 'r') as file: #percorso dei file remap +zonmean
#    lines_path_zonal_model_models_orog = file.readlines()

In [4]:
# Itero su tutte le righe dei 3 file e organizzo i dati nel dizionario
for line_name, line_path, line_path_remap in zip(lines_name_models_orog, lines_path_models_orog, lines_path_remap_models_orog): #zip per iterare sulle 3 liste contemporaneamente
    model_name = line_name.strip() #senza spazi bianchi
    #model_name è la chiave principale
    # Inizializzo il dizionario se non esiste ancora
    if model_name not in models_orog:
        models_orog[model_name] = {}
    # Aggiungo name, path, path remap come sottochiavi
    models_orog[model_name]['orog name'] = model_name
    models_orog[model_name]['orog path'] = line_path.strip()
    models_orog[model_name]['orog path remap'] = line_path_remap.strip()

In [5]:
model_name = 'ACCESS-CM2'
models_orog[model_name]['orog path remap']

'/work/users/guest/sbeltrami/land_orog/ACCESS-CM2_remapcon.nc'

In [6]:
#REMAP
for name in models_orog.keys():
    path_remap = models_orog[name]['orog path remap'] #remap
    if os.path.exists(path_remap): #Se il file esiste
        # Apro il dataset
        models_orog[name]['orog remap dataset'] = xr.open_dataset(path_remap)
        dataset_remap = models_orog[name]['orog remap dataset']
        #Seleziono la regione con le Montagne Rocciose per poter calcolare il valore massimo di altezza
        # Seleziono la parte delle lon negative
        lon_neg = dataset_remap.sel(lon=slice(235,260), lat=slice(20,60))
        lon_neg = lon_neg.assign_coords(lon=(np.where(lon_neg.lon >= 236, lon_neg.lon - 360, lon_neg.lon)))
        models_orog[name]['orog remap dataset'] = lon_neg
    else:
        print(f"Errore: Il file {path_remap} non esiste per il modello {name}")

In [11]:
#NO remap
for name in models_orog.keys():
    path = models_orog[name]['orog path']
    #if os.path.exists(path): #Se il file esiste
    # Apro il dataset
    models_orog[name]['orog dataset'] = xr.open_dataset(path)
    dataset = models_orog[name]['orog dataset']
    #Seleziono la regione con le Montagne Rocciose per poter calcolare il valore massimo di altezza
    # Seleziono la parte delle lon negative
    lon_neg = dataset.sel(lon=slice(235,360), lat=slice(20,60))
    lon_neg = lon_neg.assign_coords(lon=(np.where(lon_neg.lon >= 236, lon_neg.lon - 360, lon_neg.lon)))
    models_orog[name]['orog dataset'] = lon_neg
    #else:
    #    print(f"Errore: Il file {path} non esiste per il modello {name}")

KeyError: "'lon' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'longitude': 96, 'bnds': 2, 'latitude': 80})"

In [12]:
#Calcolo il valore massimo per ogni modello
for name in models_orog.keys():
    models_orog[name]['orog max value'] = models_orog[name]['orog remap dataset'].max().values
    print(name,models_orog[name]['orog max value'])

TaiESM1 <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.486e+03>
AWI-CM-1-1-MR <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.717e+03>
AWI-ESM-1-1-LR <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.353e+03>
CanESM5 <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.264e+03>
CMCC-CM2-HR4 <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.692e+03>
CMCC-CM2-SR5 <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.692e+03>
CMCC-ESM2 <bound method Mapping.values of <xarray.Dataset> Size: 4B
Dimensions:  ()
Data variables:
    orog     float32 4B 2.692e+03>
ACCESS-ESM1-5 <bound method Mapping.values o